In [115]:
# Import the EDA packages
import pandas as pd
import numpy as np
import seaborn as sns


from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import pairwise_distances

# Load the datasets
pd.set_option('display.max_columns', None) # Display Preference

In [14]:
df_stats_AS = pd.read_csv('NHL_Skater_AS_All_Stats_Clean.csv')
df_stats_ES = pd.read_csv('NHL_Skater_ES_All_Stats_Clean.csv')
df_stats_PP = pd.read_csv('NHL_Skater_PP_All_Stats_Clean.csv')
df_stats_PK = pd.read_csv('NHL_Skater_PK_All_Stats_Clean.csv')

In [38]:
df_stats_AS['Team'].value_counts()

Team
S.J    105
MTL    103
CBJ    101
WSH    101
VAN    100
PHI    100
TOR     98
OTT     98
VGK     98
PIT     93
CHI     93
MIN     92
WPG     91
BOS     90
STL     90
COL     88
N.J     88
NSH     88
ARI     88
T.B     86
L.A     86
EDM     84
SEA     84
NYI     84
ANA     83
FLA     81
NYR     80
BUF     80
DET     79
CAR     78
CGY     76
DAL     75
Name: count, dtype: int64

# FEATURE ENGINEERING:

### Creating a column that will indicate if that player's team made the playoffs in that respective season

## Playoff_Team Column:

In [39]:
playoff_teams_2022 = ['FLA', 'WSH', 'T.B', 'TOR', 'CAR', 'BOS', 'NYR', 'PIT',
                       'COL', 'NSH', 'STL', 'MIN', 'CGY', 'DAL', 'EDM', 'L.A',]

playoff_teams_2023 = ['FLA', 'BOS', 'TOR', 'T.B', 'CAR', 'NYI', 'N.J', 'NYR',
                      'SEA', 'COL', 'DAL', 'MIN', 'VGK', 'WPG', 'EDM', 'L.A',]

playoff_teams_2024 = ['FLA', 'T.B', 'BOS', 'TOR', 'NYR', 'WSH', 'CAR', 'NYI',
                      'DAL', 'VGK', 'COL', 'WPG', 'VAN', 'NSH', 'EDM', 'L.A',]

In [41]:
def made_playoffs(row):
    ''' made_playoffs functon is done to create the 'Playoff_Team' column in each of the game states dataframes using the .apply() method.
    Future additions would just need to create a variable for the list of playoff teams and then and a new 'elif' line for the most recent season.'''
    if row['Season'] == 2022 and row['Team'] in playoff_teams_2022:
        return 1
    elif row['Season'] == 2023 and row['Team'] in playoff_teams_2023:
        return 1
    elif row['Season'] == 2024 and row['Team'] in playoff_teams_2024:
        return 1
    else:
        return 0

In [46]:
# AS playoff teams:
df_stats_AS['Playoff_Team'] = df_stats_AS.apply(made_playoffs, axis=1)

# ES playoff teams:
df_stats_ES['Playoff_Team'] = df_stats_ES.apply(made_playoffs, axis=1)

# PP playoff teams:
df_stats_PP['Playoff_Team'] = df_stats_PP.apply(made_playoffs, axis=1)

# PK playoff teams:
df_stats_PK['Playoff_Team'] = df_stats_PK.apply(made_playoffs, axis=1)

## Player_Role Column:


#### How should the playes' roles be best account for, by line and defensive pair? Or Top line, middle six, bottom line? What about the D?

In [49]:
df_stats_AS.head(10)

,Player,Team,Position,Age,GP,TOI,Goals,Total Assists,First Assists,Second Assists,Total Points,IPP,Shots,SH%,ixG,iCF,iFF,iSCF,iHDCF,Rush Attempts,Rebounds Created,PIM,Total Penalties,Minor,Major,Misconduct,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost,Faceoffs %,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,HDCF,HDCA,HDCF%,HDGF,HDGA,HDGF%,MDCF,MDCA,MDCF%,MDGF,MDGA,MDGF%,LDCF,LDCA,LDCF%,LDGF,LDGA,LDGF%,On-Ice SH%,On-Ice SV%,PDO,Off. Zone Starts,Neu. Zone Starts,Def. Zone Starts,On The Fly Starts,Off. Zone Start %,Off. Zone Faceoffs,Neu. Zone Faceoffs,Def. Zone Faceoffs,Off. Zone Faceoff %,TOI/GP,Goals/60,Total Assists/60,First Assists/60,Second Assists/60,Total Points/60,Shots/60,ixG/60,iCF/60,iFF/60,iSCF/60,iHDCF/60,Rush Attempts/60,Rebounds Created/60,PIM/60,Total Penalties/60,Minor/60,Major/60,Misconduct/60,Penalties Drawn/60,Giveaways/60,Takeaways/60,Hits/60,Hits Taken/60,Shots Blocked/60,Faceoffs Won/60,Faceoffs Lost/60,CF/60,CA/60,FF/60,FA/60,SF/60,SA/60,GF/60,GA/60,xGF/60,xGA/60,SCF/60,SCA/60,HDCF/60,HDCA/60,HDGF/60,HDGA/60,MDCF/60,MDCA/60,MDGF/60,MDGA/60,LDCF/60,LDCA/60,LDGF/60,LDGA/60,Off. Zone Starts/60,Neu. Zone Starts/60,Def. Zone Starts/60,On The Fly Starts/60,Off. Zone Faceoffs/60,Neu. Zone Faceoffs/60,Def. Zone Faceoffs/60,Season,Age_Group,Playoff_Team
0,Zdeno Chara,NYI,D,45,72,1348.550000,2,12,8,4,14,25.45,81,2.47,4.08,172,114,30,3,1,11,85,27,20,5,2,11,45,14,125,46,86,0,0,0.00,1018,1461,41.06,784,1095,41.72,572,796,41.81,55,53,50.93,53.18,68.77,43.61,527,754,41.14,218,295,42.50,31,30,50.82,309,459,40.23,13,16,44.83,433,647,40.09,9,7,56.25,9.62,93.34,1.030,111,258,181,1126,38.01,306,368,403,43.16,18.729861,0.09,0.53,0.36,0.18,0.62,3.60,0.18,7.65,5.07,1.33,0.13,0.04,0.49,3.78,1.20,0.89,0.22,0.09,0.49,2.00,0.62,5.56,2.05,3.83,0.00,0.00,45.29,65.00,34.88,48.72,25.45,35.42,2.45,2.36,2.37,3.06,23.45,33.55,9.70,13.13,1.38,1.33,18.33,27.23,0.77,0.95,27.83,41.58,0.58,0.45,4.94,11.48,8.05,50.10,13.61,16.37,17.93,2022,Old Vet,0
1,Joe Thornton,FLA,C,43,34,377.600000,5,5,3,2,10,62.50,26,19.23,3.55,44,35,28,17,1,4,10,5,5,0,0,1,18,18,20,14,9,77,98,44.00,409,305,57.28,299,234,56.10,218,178,55.05,16,22,42.11,19.99,15.93,55.65,206,160,56.28,82,69,54.30,11,11,50.00,124,91,57.67,3,8,27.27,180,123,59.41,1,3,25.00,7.34,87.64,0.950,93,72,26,329,78.15,159,105,82,65.98,11.105882,0.79,0.79,0.48,0.32,1.59,4.13,0.56,6.99,5.56,4.45,2.70,0.16,0.64,1.59,0.79,0.79,0.00,0.00,0.16,2.86,2.86,3.18,2.22,1.43,12.24,15.57,64.99,48.46,47.51,37.18,34.64,28.28,2.54,3.50,3.18,2.53,32.73,25.42,13.03,10.96,1.75,1.75,26.27,19.28,0.64,1.69,41.31,28.23,0.23,0.69,14.78,11.44,4.13,52.28,25.26,16.68,13.03,2022,Old Vet,1
2,Jason Spezza,TOR,C,39,71,762.966667,12,13,9,4,25,67.57,97,12.37,9.20,175,131,83,33,8,13,26,13,13,0,0,6,31,32,31,41,22,305,272,52.86,800,629,55.98,598,498,54.56,441,358,55.19,37,28,56.92,42.06,32.48,56.43,444,309,58.96,183,122,60.00,19,14,57.58,261,187,58.26,11,10,52.38,329,284,53.67,7,3,70.00,8.39,92.18,1.006,183,130,96,726,65.59,309,195,195,61.31,10.746009,0.94,1.02,0.71,0.31,1.97,7.63,0.72,13.76,10.30,6.53,2.60,0.63,1.02,2.04,1.02,1.02,0.00,0.00,0.47,2.44,2.52,2.44,3.22,1.73,23.99,21.39,62.91,49.46,47.03,39.16,34.68,28.15,2.91,2.20,3.31,2.55,34.92,24.30,14.39,9.59,1.49,1.10,27.37,19.61,1.15,1.05,37.37,32.26,0.80,0.34,14.39,10.22,7.55,57.09,24.30,15.33,15.33,2022,Old Vet,1
3,Duncan Keith,EDM,D,39,64,1262.516667,1,20,10,10,21,34.43,79,1.27,3.42,175,112,33,4,5,9,22,11,11,0,0,2,64,36,22,60,107,0,0,0.00,1164,1206,49.11,853,916,48.22,641,685,48.34,61,57,51.69,55.59,61.81,47.35,545,655,45.42,224,251,47.16,44,33,57.14,321,404,44.28,11,15,42.31,542,498,52.12,6,9,40.00,9.52,91.68,1.012,142,259,227,862,38.48,360,373,446,44.67,19.726823,0.05,0.95,0.48,0.48,1.00,3.75,0.16,8.32,5.32,1.57,0.19,0.24,0.43,1.05,0.52,0.52,0.00,0.00,0.10,3.04,1.71,1.05,2.85,5.09,0.00,0.00,55.32,57.31,40.54,43.53,30.46,32.55,2.90,2.71,2.64,2.94,25.90,31.13,10.65,11.93,2.09,1.57,20.34,25.60,0.70,0.95,37.21,34.19,0.

### Dropping the 3 Penalty type columns for to just have Total Penalties and Penalties Drawn. :

In [50]:
penalty_cols = ['Minor', 'Major', 'Misconduct', 'Minor/60', 'Major/60', 'Misconduct/60',]
df_stats_AS.drop(columns=penalty_cols, inplace=True)
df_stats_ES.drop(columns=penalty_cols, inplace=True)
df_stats_PP.drop(columns=penalty_cols, inplace=True)
df_stats_PK.drop(columns=penalty_cols, inplace=True)

### Dropping the 2 Assist type columns of the totals and rates to have just total assists and total assists/60: 

In [52]:
assist_drop_cols = ['First Assists', 'Second Assists', 'First Assists/60', 'Second Assists/60']
df_stats_AS.drop(columns=assist_drop_cols, inplace=True)
df_stats_ES.drop(columns=assist_drop_cols, inplace=True)
df_stats_PP.drop(columns=assist_drop_cols, inplace=True)
df_stats_PK.drop(columns=assist_drop_cols, inplace=True)

In [53]:
df_stats_AS.head()

,Player,Team,Position,Age,GP,TOI,Goals,Total Assists,Total Points,IPP,Shots,SH%,ixG,iCF,iFF,iSCF,iHDCF,Rush Attempts,Rebounds Created,PIM,Total Penalties,Penalties Drawn,Giveaways,Takeaways,Hits,Hits Taken,Shots Blocked,Faceoffs Won,Faceoffs Lost,Faceoffs %,CF,CA,CF%,FF,FA,FF%,SF,SA,SF%,GF,GA,GF%,xGF,xGA,xGF%,SCF,SCA,SCF%,HDCF,HDCA,HDCF%,HDGF,HDGA,HDGF%,MDCF,MDCA,MDCF%,MDGF,MDGA,MDGF%,LDCF,LDCA,LDCF%,LDGF,LDGA,LDGF%,On-Ice SH%,On-Ice SV%,PDO,Off. Zone Starts,Neu. Zone Starts,Def. Zone Starts,On The Fly Starts,Off. Zone Start %,Off. Zone Faceoffs,Neu. Zone Faceoffs,Def. Zone Faceoffs,Off. Zone Faceoff %,TOI/GP,Goals/60,Total Assists/60,Total Points/60,Shots/60,ixG/60,iCF/60,iFF/60,iSCF/60,iHDCF/60,Rush Attempts/60,Rebounds Created/60,PIM/60,Total Penalties/60,Penalties Drawn/60,Giveaways/60,Takeaways/60,Hits/60,Hits Taken/60,Shots Blocked/60,Faceoffs Won/60,Faceoffs Lost/60,CF/60,CA/60,FF/60,FA/60,SF/60,SA/60,GF/60,GA/60,xGF/60,xGA/60,SCF/60,SCA/60,HDCF/60,HDCA/60,HDGF/60,HDGA/60,MDCF/60,MDCA/60,MDGF/60,MDGA/60,LDCF/60,LDCA/60,LDGF/60,LDGA/60,Off. Zone Starts/60,Neu. Zone Starts/60,Def. Zone Starts/60,On The Fly Starts/60,Off. Zone Faceoffs/60,Neu. Zone Faceoffs/60,Def. Zone Faceoffs/60,Season,Age_Group,Playoff_Team
0,Zdeno Chara,NYI,D,45,72,1348.550000,2,12,14,25.45,81,2.47,4.08,172,114,30,3,1,11,85,27,11,45,14,125,46,86,0,0,0.00,1018,1461,41.06,784,1095,41.72,572,796,41.81,55,53,50.93,53.18,68.77,43.61,527,754,41.14,218,295,42.50,31,30,50.82,309,459,40.23,13,16,44.83,433,647,40.09,9,7,56.25,9.62,93.34,1.030,111,258,181,1126,38.01,306,368,403,43.16,18.729861,0.09,0.53,0.62,3.60,0.18,7.65,5.07,1.33,0.13,0.04,0.49,3.78,1.20,0.49,2.00,0.62,5.56,2.05,3.83,0.00,0.00,45.29,65.00,34.88,48.72,25.45,35.42,2.45,2.36,2.37,3.06,23.45,33.55,9.70,13.13,1.38,1.33,18.33,27.23,0.77,0.95,27.83,41.58,0.58,0.45,4.94,11.48,8.05,50.10,13.61,16.37,17.93,2022,Old Vet,0
1,Joe Thornton,FLA,C,43,34,377.600000,5,5,10,62.50,26,19.23,3.55,44,35,28,17,1,4,10,5,1,18,18,20,14,9,77,98,44.00,409,305,57.28,299,234,56.10,218,178,55.05,16,22,42.11,19.99,15.93,55.65,206,160,56.28,82,69,54.30,11,11,50.00,124,91,57.67,3,8,27.27,180,123,59.41,1,3,25.00,7.34,87.64,0.950,93,72,26,329,78.15,159,105,82,65.98,11.105882,0.79,0.79,1.59,4.13,0.56,6.99,5.56,4.45,2.70,0.16,0.64,1.59,0.79,0.16,2.86,2.86,3.18,2.22,1.43,12.24,15.57,64.99,48.46,47.51,37.18,34.64,28.28,2.54,3.50,3.18,2.53,32.73,25.42,13.03,10.96,1.75,1.75,26.27,19.28,0.64,1.69,41.31,28.23,0.23,0.69,14.78,11.44,4.13,52.28,25.26,16.68,13.03,2022,Old Vet,1
2,Jason Spezza,TOR,C,39,71,762.966667,12,13,25,67.57,97,12.37,9.20,175,131,83,33,8,13,26,13,6,31,32,31,41,22,305,272,52.86,800,629,55.98,598,498,54.56,441,358,55.19,37,28,56.92,42.06,32.48,56.43,444,309,58.96,183,122,60.00,19,14,57.58,261,187,58.26,11,10,52.38,329,284,53.67,7,3,70.00,8.39,92.18,1.006,183,130,96,726,65.59,309,195,195,61.31,10.746009,0.94,1.02,1.97,7.63,0.72,13.76,10.30,6.53,2.60,0.63,1.02,2.04,1.02,0.47,2.44,2.52,2.44,3.22,1.73,23.99,21.39,62.91,49.46,47.03,39.16,34.68,28.15,2.91,2.20,3.31,2.55,34.92,24.30,14.39,9.59,1.49,1.10,27.37,19.61,1.15,1.05,37.37,32.26,0.80,0.34,14.39,10.22,7.55,57.09,24.30,15.33,15.33,2022,Old Vet,1
3,Duncan Keith,EDM,D,39,64,1262.516667,1,20,21,34.43,79,1.27,3.42,175,112,33,4,5,9,22,11,2,64,36,22,60,107,0,0,0.00,1164,1206,49.11,853,916,48.22,641,685,48.34,61,57,51.69,55.59,61.81,47.35,545,655,45.42,224,251,47.16,44,33,57.14,321,404,44.28,11,15,42.31,542,498,52.12,6,9,40.00,9.52,91.68,1.012,142,259,227,862,38.48,360,373,446,44.67,19.726823,0.05,0.95,1.00,3.75,0.16,8.32,5.32,1.57,0.19,0.24,0.43,1.05,0.52,0.10,3.04,1.71,1.05,2.85,5.09,0.00,0.00,55.32,57.31,40.54,43.53,30.46,32.55,2.90,2.71,2.64,2.94,25.90,31.13,10.65,11.93,2.09,1.57,20.34,25.60,0.70,0.95,37.21,34.19,0.41,0.62,6.75,12.31,10.79,40.97,17.11,17.73,21.20,2022,Old Vet,1
4,Ryan Suter,DAL,D,37,82,1939.483333,7,25,32,35.16,118,5.93,6.60,265,183,79,13,5,17,40,20,5,46,19,46,64,83,0,0,0.00,1904,1770,51.82,1403,1377,50.47,1006,997,50.22,91,84,52.00,103.23,88.87,53.74,1016,8

### Determining My new ZRF (Zach Rosenthal For metric). This metric is a combination of shots and blocked shots by a player. This is done by doing ((CF - FF) + Shots). Shots is made up of goals and shots on net. The rationale is that a miss (as account for in the Fenwick and Corsi scores) will always be a miss. A blocked shot on the other hand as a chance to be a goal or shot on net (and at the very least much more likely than an outright miss) even though hitting the posts are considered misses. Undoubtedly, shooting into an opposing player may lead to a scoring chance for the other team, I am considering that a blocked shot is more an indication of a well positioned defender than on an error with the shooter and should therefore not be counted as a detriment. 

## The ZR For Column (ZRF) and Individual ZR For Column iZRF :


In [80]:
#AS iZRF:
df_stats_AS['iZRF'] = ((df_stats_AS['iCF'] - df_stats_AS['iFF']) + df_stats_AS['Shots'])
df_stats_ES['iZRF'] = ((df_stats_ES['iCF'] - df_stats_ES['iFF']) + df_stats_ES['Shots'])
df_stats_PP['iZRF'] = ((df_stats_PP['iCF'] - df_stats_PP['iFF']) + df_stats_PP['Shots'])
df_stats_PK['iZRF'] = ((df_stats_PK['iCF'] - df_stats_PK['iFF']) + df_stats_PK['Shots'])

#AS ZRF:
df_stats_AS['ZRF'] = ((df_stats_AS['CF'] - df_stats_AS['FF']) + df_stats_AS['SF'])
df_stats_ES['ZRF'] = ((df_stats_ES['CF'] - df_stats_ES['FF']) + df_stats_ES['SF'])
df_stats_PP['ZRF'] = ((df_stats_PP['CF'] - df_stats_PP['FF']) + df_stats_PP['SF'])
df_stats_PK['ZRF'] = ((df_stats_PK['CF'] - df_stats_PK['FF']) + df_stats_PK['SF'])

#AS ZRA:
df_stats_AS['ZRA'] = ((df_stats_AS['CA'] - df_stats_AS['FA'] + df_stats_AS['SA']))
df_stats_ES['ZRA'] = ((df_stats_ES['CA'] - df_stats_ES['FA'] + df_stats_ES['SA']))
df_stats_PP['ZRA'] = ((df_stats_PP['CA'] - df_stats_PP['FA'] + df_stats_PP['SA']))
df_stats_PK['ZRA'] = ((df_stats_PK['CA'] - df_stats_PK['FA'] + df_stats_PK['SA']))

#AS ZRF%:
df_stats_AS['ZRF%'] = ((df_stats_AS['ZRF'] * 100) / (df_stats_AS['ZRF'] + df_stats_AS['ZRA']))
df_stats_ES['ZRF%'] = ((df_stats_ES['ZRF'] * 100) / (df_stats_ES['ZRF'] + df_stats_ES['ZRA']))
df_stats_PP['ZRF%'] = ((df_stats_PP['ZRF'] * 100) / (df_stats_PP['ZRF'] + df_stats_PP['ZRA']))
df_stats_PK['ZRF%'] = ((df_stats_PK['ZRF'] * 100) / (df_stats_PK['ZRF'] + df_stats_PK['ZRA']))

#AS iZRF/60:
df_stats_AS['iZRF/60'] = (df_stats_AS['iZRF'] * (60 / (df_stats_AS['TOI'])))
df_stats_ES['iZRF/60'] = (df_stats_ES['iZRF'] * (60 / (df_stats_ES['TOI'])))
df_stats_PP['iZRF/60'] = (df_stats_PP['iZRF'] * (60 / (df_stats_PP['TOI'])))
df_stats_PK['iZRF/60'] = (df_stats_PK['iZRF'] * (60 / (df_stats_PK['TOI'])))

#AS ZRF/60:
df_stats_AS['ZRF/60'] = (df_stats_AS['ZRF'] * (60 / (df_stats_AS['TOI'])))
df_stats_ES['ZRF/60'] = (df_stats_ES['ZRF'] * (60 / (df_stats_ES['TOI'])))
df_stats_PP['ZRF/60'] = (df_stats_PP['ZRF'] * (60 / (df_stats_PP['TOI'])))
df_stats_PK['ZRF/60'] = (df_stats_PK['ZRF'] * (60 / (df_stats_PK['TOI'])))

#AS ZRA/60:
df_stats_AS['ZRA/60'] = (df_stats_AS['ZRA'] * (60 / (df_stats_AS['TOI'])))
df_stats_ES['ZRA/60'] = (df_stats_ES['ZRA'] * (60 / (df_stats_ES['TOI'])))
df_stats_PP['ZRA/60'] = (df_stats_PP['ZRA'] * (60 / (df_stats_PP['TOI'])))
df_stats_PK['ZRA/60'] = (df_stats_PK['ZRA'] * (60 / (df_stats_PK['TOI'])))


## Group Teams tpgether by season to quickly get a feel for how the ZRF and ZRF% look based on making the playoffs:

#### Step 1: Group by team:

In [99]:
NHL_teams_list = (list(set(df_stats_AS['Team']))) # 32 Teams is correct
seasons = [2022, 2023, 2024]

def get_team_stats(gamestate_df, team, season):
    team_stats = gamestate_df.loc[
        (gamestate_df['Team'] == team) &
        (gamestate_df['Season'] == season)
    ]
    return team_stats

## EXPLORING COLUMN CORRELATION:

In [144]:
#AS Correlations - Playoff Team
AS_numeric_df = df_stats_AS.select_dtypes(include=['number'])
AS_playoff_corr = AS_numeric_df.corr()['Playoff_Team'].sort_values(ascending=False)
AS_playoff_corr

Playoff_Team    1.000000
xGF%            0.254095
SCF%            0.245566
HDCF%           0.235359
FF%             0.234913
                  ...   
FA/60          -0.194738
xGA/60         -0.194921
MDCA/60        -0.196673
CA/60          -0.201513
SCA/60         -0.217620
Name: Playoff_Team, Length: 137, dtype: float64

In [148]:
#AS Correlations - Goals
AS_numeric_df = df_stats_AS.select_dtypes(include=['number'])
AS_playoff_corr = AS_numeric_df.corr()['Goals'].sort_values(ascending=False)
AS_playoff_corr

Goals                   1.000000
iSCF                    0.937260
ixG                     0.935904
Total Points            0.916356
Shots                   0.885923
                          ...   
CA/60                  -0.264782
Hits Taken/60          -0.267461
ZRA/60                 -0.267713
Shots Blocked/60       -0.306792
On The Fly Starts/60   -0.550317
Name: Goals, Length: 137, dtype: float64

In [145]:
#ES Correlations - Playoff Team
ES_numeric_df = df_stats_ES.select_dtypes(include=['number'])
ES_playoff_corr = ES_numeric_df.corr()['Playoff_Team'].sort_values(ascending=False)
ES_playoff_corr

Playoff_Team    1.000000
xGF%            0.309192
SCF%            0.300221
CF%             0.287703
FF%             0.286358
                  ...   
FA/60          -0.214150
MDCA/60        -0.215126
ZRA/60         -0.217599
CA/60          -0.230330
SCA/60         -0.234901
Name: Playoff_Team, Length: 137, dtype: float64

In [149]:
#ES Correlations - Goals
ES_numeric_df = df_stats_ES.select_dtypes(include=['number'])
ES_playoff_corr = ES_numeric_df.corr()['Goals'].sort_values(ascending=False)
ES_playoff_corr

Goals                   1.000000
iSCF                    0.915820
ixG                     0.914623
Total Points            0.909759
iHDCF                   0.863042
                          ...   
ZRA/60                 -0.097690
Hits/60                -0.212947
Hits Taken/60          -0.229670
Shots Blocked/60       -0.271644
On The Fly Starts/60   -0.438344
Name: Goals, Length: 137, dtype: float64

In [146]:
#PP Correlations - Playoff Team:
PP_numeric_df = df_stats_PP.select_dtypes(include=['number'])
PP_playoff_corr = PP_numeric_df.corr()['Playoff_Team'].sort_values(ascending=False)
PP_playoff_corr

Playoff_Team           1.000000
Age                    0.129681
GP                     0.125784
HDGF                   0.121787
GF                     0.118395
                         ...   
Off. Zone Starts/60   -0.039664
GA/60                 -0.040264
GA                    -0.046600
HDGA/60               -0.050121
MDGA                  -0.067186
Name: Playoff_Team, Length: 137, dtype: float64

In [150]:
#PP Correlations - Goals:
PP_numeric_df = df_stats_PP.select_dtypes(include=['number'])
PP_playoff_corr = PP_numeric_df.corr()['Goals'].sort_values(ascending=False)
PP_playoff_corr

Goals                    1.000000
ixG                      0.896316
iSCF                     0.896136
Shots                    0.884481
iFF                      0.883125
                           ...   
LDCA/60                 -0.056638
Off. Zone Starts/60     -0.068500
Neu. Zone Faceoffs/60   -0.073938
Neu. Zone Starts/60     -0.074109
On The Fly Starts/60    -0.216562
Name: Goals, Length: 137, dtype: float64

In [147]:
# PK Correlations - Playoff Team:
PK_numeric_df = df_stats_PK.select_dtypes(include=['number'])
PK_playoff_corr = PK_numeric_df.corr()['Playoff_Team'].sort_values(ascending=False)
PK_playoff_corr

Playoff_Team    1.000000
Age             0.133908
GP              0.130440
xGF             0.121151
GF              0.117553
                  ...   
HDGA/60        -0.060145
GA/60          -0.062925
HDCA/60        -0.066880
SCA/60         -0.070270
xGA/60         -0.077714
Name: Playoff_Team, Length: 137, dtype: float64

In [151]:
# PK Correlations - Goals:
PK_numeric_df = df_stats_PK.select_dtypes(include=['number'])
PK_playoff_corr = PK_numeric_df.corr()['Goals'].sort_values(ascending=False)
PK_playoff_corr

Goals                    1.000000
Total Points             0.809569
ixG                      0.714193
iSCF                     0.687128
iHDCF                    0.671435
                           ...   
Off. Zone Starts/60     -0.038916
Off. Zone Faceoffs/60   -0.043893
Off. Zone Start %       -0.076218
Off. Zone Faceoff %     -0.084486
On The Fly Starts/60    -0.116145
Name: Goals, Length: 137, dtype: float64

## Trying to determin a metric for positive and negative impact:

#### I'm thinking about using the non-aggregated statistic columns as a starting point. By non-aggregated, I mean metrics like 'IPP' which is calculated by doing ** Total Points / Goals For ** Even though this would be generating a score for the entire season, it would be identical from game to game as well. It might make sense to have the whole thing divided by TOI (or TOI/GP). 

#### I'm first splitting the cleaned datasets by position but keeping all seasons together for now.

In [47]:
df_stats_AS_C = df_stats_AS.loc[df_stats_AS['Position'] == 'C']
df_stats_AS_D = df_stats_AS.loc[df_stats_AS['Position'] == 'D']
df_stats_AS_W = df_stats_AS.loc[(df_stats_AS['Position'] == 'L') | (df_stats_AS['Position'] == 'R')]

## Checking out the MoneyPuck stats - EXPERIMENT:

In [154]:
MP_NHL_all_teams_stats = pd.read_csv('MoneyPuck_NHL_all_teams.csv') #This is really just game stats
MP_NHL_all_teams_stats.head()

,team,season,name,gameId,playerTeam,opposingTeam,home_or_away,gameDate,position,situation,xGoalsPercentage,corsiPercentage,fenwickPercentage,iceTime,xOnGoalFor,xGoalsFor,xReboundsFor,xFreezeFor,xPlayStoppedFor,xPlayContinuedInZoneFor,xPlayContinuedOutsideZoneFor,flurryAdjustedxGoalsFor,scoreVenueAdjustedxGoalsFor,flurryScoreVenueAdjustedxGoalsFor,shotsOnGoalFor,missedShotsFor,blockedShotAttemptsFor,shotAttemptsFor,goalsFor,reboundsFor,reboundGoalsFor,freezeFor,playStoppedFor,playContinuedInZoneFor,playContinuedOutsideZoneFor,savedShotsOnGoalFor,savedUnblockedShotAttemptsFor,penaltiesFor,penalityMinutesFor,faceOffsWonFor,hitsFor,takeawaysFor,giveawaysFor,lowDangerShotsFor,mediumDangerShotsFor,highDangerShotsFor,lowDangerxGoalsFor,mediumDangerxGoalsFor,highDangerxGoalsFor,lowDangerGoalsFor,mediumDangerGoalsFor,highDangerGoalsFor,scoreAdjustedShotsAttemptsFor,unblockedShotAttemptsFor,scoreAdjustedUnblockedShotAttemptsFor,dZoneGiveawaysFor,xGoalsFromxReboundsOfShotsFor,xGoalsFromActualReboundsOfShotsFor,reboundxGoalsFor,totalShotCreditFor,scoreAdjustedTotalShotCreditFor,scoreFlurryAdjustedTotalShotCreditFor,xOnGoalAgainst,xGoalsAgainst,xReboundsAgainst,xFreezeAgainst,xPlayStoppedAgainst,xPlayContinuedInZoneAgainst,xPlayContinuedOutsideZoneAgainst,flurryAdjustedxGoalsAgainst,scoreVenueAdjustedxGoalsAgainst,flurryScoreVenueAdjustedxGoalsAgainst,shotsOnGoalAgainst,missedShotsAgainst,blockedShotAttemptsAgainst,shotAttemptsAgainst,goalsAgainst,reboundsAgainst,reboundGoalsAgainst,freezeAgainst,playStoppedAgainst,playContinuedInZoneAgainst,playContinuedOutsideZoneAgainst,savedShotsOnGoalAgainst,savedUnblockedShotAttemptsAgainst,penaltiesAgainst,penalityMinutesAgainst,faceOffsWonAgainst,hitsAgainst,takeawaysAgainst,giveawaysAgainst,lowDangerShotsAgainst,mediumDangerShotsAgainst,highDangerShotsAgainst,lowDangerxGoalsAgainst,mediumDangerxGoalsAgainst,highDangerxGoalsAgainst,lowDangerGoalsAgainst,mediumDangerGoalsAgainst,highDangerGoalsAgainst,scoreAdjustedShotsAttemptsAgainst,unblockedShotAttemptsAgainst,scoreAdjustedUnblockedShotAttemptsAgainst,dZoneGiveawaysAgainst,xGoalsFromxReboundsOfShotsAgainst,xGoalsFromActualReboundsOfShotsAgainst,reboundxGoalsAgainst,totalShotCreditAgainst,scoreAdjustedTotalShotCreditAgainst,scoreFlurryAdjustedTotalShotCreditAgainst,playoffGame
0,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,other,0.0000,0.0000,0.0000,42.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.000,0.0,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.689,0.019,0.069,0.164,0.020,0.521,0.207,0.019,0.019,0.019,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.019,0.000,0.000,0.0,0.0,0.0,1.00,1.0,1.000,0.0,0.017,0.000,0.000,0.037,0.037,0.037,0
1,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,all,0.4596,0.6408,0.6310,3600.0,37.733,2.287,2.842,8.155,1.134,22.530,16.054,2.249,2.396,2.357,41.0,12.0,13.0,66.0,2.0,1.0,0.0,14.0,2.0,18.0,16.0,39.0,51.0,4.0,8.0,30.0,16.0,19.0,8.0,44.0,8.0,1.0,1.040,0.811,0.435,1.0,0.0,1.0,68.292,53.0,54.697,4.0,0.641,0.014,0.029,2.899,3.030,2.980,22.880,2.689,1.780,4.436,0.639,12.626,8.830,2.524,2.607,2.450,21.0,10.0,6.0,37.0,1.0,1.0,0.0,5.0,1.0,8.0,15.0,20.0,30.0,7.0,14.0,32.0,17.0,11.0,7.0,16.0,12.0,3.0,0.599,1.278,0.811,1.0,0.0,0.0,36.14,31.0,30.369,5.0,0.396,0.168,0.168,2.917,2.833,2.714,0
2,NYR,2008,NYR,2008020001,NYR,T.B,AWAY,20081004,Team Level,5on5,0.4857,0.6429,0.6364,2283.0,24.802,1.693,1.724,5.513,0.739,14.650,10.681,1.656,1.803,1.764,28.0,7.0,10.0,45.0,1.0,0.0,0.0,10.0,1.0,14.0,9.0,27.0,34.0,3.0,6.0,19.0,13.0,13.0,7.0,27.0,8.0,0.0,0.882,0.811,0.000,1.0,0.0,0.0,47.292,35.0,36.697,4.0,0.355,0.000,0.000,2.048,2.180,2.130,14.729,1.793,1.117,3.001,0.406,7.737,5.946,1.633,1.711,1.559,11.0,9.0,5.0,25.0,1.0,1.0,0.0,4.0,1.0,4.0,9.0,10.0,19.0,7.0,14.0,18.0,13.0,9.0,5.0,12.0,6.0,2.0,0.498,0.689,0.606,1.0,0.0,0.0,24.14,20.0,19.369

#### hile that's not exactly what I'm looking for, I do think there are some interesting connections that can be made based on wins and losses based on travel time and other things like that. It is good have all these details in my data sets anyway. But right now, I want to explore other things.

In [185]:
MP_skaters_2021_2022 = pd.read_csv('MoneyPuck_skaters_2021_2022.csv')
MP_skaters_2022_2023 = pd.read_csv('MoneyPuck_skaters_2022_2023.csv')
MP_skaters_2023_2024 = pd.read_csv('MoneyPuck_skaters_2023_2024.csv')

## Creating the MoneyPuck Gamestate dataframes

In [186]:
# 2021-2022 Regular Season
MP_all_situations_2021_2022_df = MP_skaters_2021_2022.loc[MP_skaters_2021_2022['situation'] == 'all']
MP_5on5_2021_2022_df = MP_skaters_2021_2022.loc[MP_skaters_2021_2022['situation'] == '5on5']
MP_4on5_2021_2022_df = MP_skaters_2021_2022.loc[MP_skaters_2021_2022['situation'] == '4on5']
MP_5on4_2021_2022_df = MP_skaters_2021_2022.loc[MP_skaters_2021_2022['situation'] == '5on4']
MP_other_situations_2021_2022_df = MP_skaters_2021_2022.loc[MP_skaters_2021_2022['situation'] == 'other']

#2022-2023 Regular Season
MP_all_situations_2022_2023_df = MP_skaters_2022_2023.loc[MP_skaters_2022_2023['situation'] == 'all']
MP_5on5_2022_2023_df = MP_skaters_2022_2023.loc[MP_skaters_2022_2023['situation'] == '5on5']
MP_4on5_2022_2023_df = MP_skaters_2022_2023.loc[MP_skaters_2022_2023['situation'] == '4on5']
MP_5on4_2022_2023_df = MP_skaters_2022_2023.loc[MP_skaters_2022_2023['situation'] == '5on4']
MP_other_situations_2022_2023_df = MP_skaters_2022_2023.loc[MP_skaters_2022_2023['situation'] == 'other']

#2023-2024 Regular Season
MP_all_situations_2023_2024_df = MP_skaters_2023_2024.loc[MP_skaters_2023_2024['situation'] == 'all']
MP_5on5_2023_2024_df = MP_skaters_2023_2024.loc[MP_skaters_2023_2024['situation'] == '5on5']
MP_4on5_2023_2024_df = MP_skaters_2023_2024.loc[MP_skaters_2023_2024['situation'] == '4on5']
MP_5on4_2023_2024_df = MP_skaters_2023_2024.loc[MP_skaters_2023_2024['situation'] == '5on4']
MP_other_situations_2023_2024_df = MP_skaters_2023_2024.loc[MP_skaters_2023_2024['situation'] == 'other']


In [ ]:
# Resetting All the indeces so that it makes it easier for the recommender. 
# Note that each player has a unique ID regardless of season, so make sure to remember that if things go wrong

# # All Situations:
# MP_all_situations_2021_2022_df.set_index(keys='playerId', drop=True, inplace=True)
# MP_all_situations_2022_2023_df.set_index(keys='playerId', drop=True, inplace=True)
# MP_all_situations_2023_2024_df.set_index(keys='playerId', drop=True, inplace=True)
# # 5on5:
# MP_5on5_2021_2022_df.set_index(keys='playerId', drop=True, inplace=True)
# MP_5on5_2022_2023_df.set_index(keys='playerId', drop=True, inplace=True)
# MP_5on5_2023_2024_df.set_index(keys='playerId', drop=True, inplace=True)
# # 4on5:
# MP_4on5_2021_2022_df.set_index(keys='playerId', drop=True, inplace=True)
# MP_4on5_2022_2023_df.set_index(keys='playerId', drop=True, inplace=True)
# MP_4on5_2023_2024_df.set_index(keys='playerId', drop=True, inplace=True)
# #5on4:
# MP_5on4_2021_2022_df.set_index(keys='playerId', drop=True, inplace=True)
# MP_5on4_2022_2023_df.set_index(keys='playerId', drop=True, inplace=True)
# MP_5on4_2023_2024_df.set_index(keys='playerId', drop=True, inplace=True)
# #Other Situations:
# MP_other_situations_2021_2022_df.set_index(keys='playerId', drop=True, inplace=True)
# MP_other_situations_2022_2023_df.set_index(keys='playerId', drop=True, inplace=True)
# MP_other_situations_2023_2024_df.set_index(keys='playerId', drop=True, inplace=True)

In [187]:
#Saving the new MoneyPuck Datafranes as CSVs:
# All Situations:
MP_all_situations_2021_2022_df.to_csv('MoneyPuck_all_situations_2021_2022.csv', index=0)
MP_all_situations_2022_2023_df.to_csv('MoneyPuck_all_situations_2022_2023.csv', index=0)
MP_all_situations_2023_2024_df.to_csv('MoneyPuck_all_situations_2023_2024.csv', index=0)
# 5on5:
MP_5on5_2021_2022_df.to_csv('MoneyPuck_5on5_2021_2022.csv', index=0)
MP_5on5_2022_2023_df.to_csv('MoneyPuck_5on5_2022_2023.csv', index=0)
MP_5on5_2023_2024_df.to_csv('MoneyPuck_5on5_2023_2024.csv', index=0)
# 4on5:
MP_4on5_2021_2022_df.to_csv('MoneyPuck_4on5_2021_2022.csv', index=0)
MP_4on5_2022_2023_df.to_csv('MoneyPuck_4on5_2022_2023.csv', index=0)
MP_4on5_2023_2024_df.to_csv('MoneyPuck_4on5_2023_2024.csv', index=0)
#5on4:
MP_5on4_2021_2022_df.to_csv('MoneyPuck_5on4_2021_2022.csv', index=0)
MP_5on4_2022_2023_df.to_csv('MoneyPuck_5on4_2022_2023.csv', index=0)
MP_5on4_2023_2024_df.to_csv('MoneyPuck_5on4_2023_2024.csv', index=0)
#Other Situations:
MP_other_situations_2021_2022_df.to_csv('MoneyPuck_other_situations_2021_2022.csv', index=0)
MP_other_situations_2022_2023_df.to_csv('MoneyPuck_other_situations_2022_2023.csv', index=0)
MP_other_situations_2023_2024_df.to_csv('MoneyPuck_other_situations_2023_2024.csv', index=0)

In [184]:
MP_all_situations_2022_2023_df

,playerId,season,name,team,position,situation,games_played,icetime,shifts,gameScore,onIce_xGoalsPercentage,offIce_xGoalsPercentage,onIce_corsiPercentage,offIce_corsiPercentage,onIce_fenwickPercentage,offIce_fenwickPercentage,iceTimeRank,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_xFreeze,I_F_xPlayStopped,I_F_xPlayContinuedInZone,I_F_xPlayContinuedOutsideZone,I_F_flurryAdjustedxGoals,I_F_scoreVenueAdjustedxGoals,I_F_flurryScoreVenueAdjustedxGoals,I_F_primaryAssists,I_F_secondaryAssists,I_F_shotsOnGoal,I_F_missedShots,I_F_blockedShotAttempts,I_F_shotAttempts,I_F_points,I_F_goals,I_F_rebounds,I_F_reboundGoals,I_F_freeze,I_F_playStopped,I_F_playContinuedInZone,I_F_playContinuedOutsideZone,I_F_savedShotsOnGoal,I_F_savedUnblockedShotAttempts,penalties,I_F_penalityMinutes,I_F_faceOffsWon,I_F_hits,I_F_takeaways,I_F_giveaways,I_F_lowDangerShots,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_scoreAdjustedShotsAttempts,I_F_unblockedShotAttempts,I_F_scoreAdjustedUnblockedShotAttempts,I_F_dZoneGiveaways,I_F_xGoalsFromxReboundsOfShots,I_F_xGoalsFromActualReboundsOfShots,I_F_reboundxGoals,I_F_xGoals_with_earned_rebounds,I_F_xGoals_with_earned_rebounds_scoreAdjusted,I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,I_F_shifts,I_F_oZoneShiftStarts,I_F_dZoneShiftStarts,I_F_neutralZoneShiftStarts,I_F_flyShiftStarts,I_F_oZoneShiftEnds,I_F_dZoneShiftEnds,I_F_neutralZoneShiftEnds,I_F_flyShiftEnds,faceoffsWon,faceoffsLost,timeOnBench,penalityMinutes,penalityMinutesDrawn,penaltiesDrawn,shotsBlockedByPlayer,OnIce_F_xOnGoal,OnIce_F_xGoals,OnIce_F_flurryAdjustedxGoals,OnIce_F_scoreVenueAdjustedxGoals,OnIce_F_flurryScoreVenueAdjustedxGoals,OnIce_F_shotsOnGoal,OnIce_F_missedShots,OnIce_F_blockedShotAttempts,OnIce_F_shotAttempts,OnIce_F_goals,OnIce_F_rebounds,OnIce_F_reboundGoals,OnIce_F_lowDangerShots,OnIce_F_mediumDangerShots,OnIce_F_highDangerShots,OnIce_F_lowDangerxGoals,OnIce_F_mediumDangerxGoals,OnIce_F_highDangerxGoals,OnIce_F_lowDangerGoals,OnIce_F_mediumDangerGoals,OnIce_F_highDangerGoals,OnIce_F_scoreAdjustedShotsAttempts,OnIce_F_unblockedShotAttempts,OnIce_F_scoreAdjustedUnblockedShotAttempts,OnIce_F_xGoalsFromxReboundsOfShots,OnIce_F_xGoalsFromActualReboundsOfShots,OnIce_F_reboundxGoals,OnIce_F_xGoals_with_earned_rebounds,OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OnIce_A_xOnGoal,OnIce_A_xGoals,OnIce_A_flurryAdjustedxGoals,OnIce_A_scoreVenueAdjustedxGoals,OnIce_A_flurryScoreVenueAdjustedxGoals,OnIce_A_shotsOnGoal,OnIce_A_missedShots,OnIce_A_blockedShotAttempts,OnIce_A_shotAttempts,OnIce_A_goals,OnIce_A_rebounds,OnIce_A_reboundGoals,OnIce_A_lowDangerShots,OnIce_A_mediumDangerShots,OnIce_A_highDangerShots,OnIce_A_lowDangerxGoals,OnIce_A_mediumDangerxGoals,OnIce_A_highDangerxGoals,OnIce_A_lowDangerGoals,OnIce_A_mediumDangerGoals,OnIce_A_highDangerGoals,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts
1,8471817,2022,Ryan Reaves,MIN,R,all,73,40825.0,889.0,8.58,0.45,0.52,0.45,0.52,0.46,0.53,814.0,49.47,6.70,3.80,11.20,1.72,26.57,19.02,6.55,6.77,6.62,5.0,5.0,48.0,21.0,10.0,79.0,15.0,5.0,5.0,1.0,7.0,1.0,24.0,27.0,43.0,64.0,11.0,43.0,4.0,196.0,15.0,16.0,33.0,29.0,7.0,1.16,3.58,1.96,0.0,4.0,1.0,80.30,69.0,69.70,10.0,0.80,1.15,1.90,5.60,5.68,5.59,889.0,70.0,91.0,141.0,587.0,138.0,140.0,98.0,513.0,4.0,11.0,227036.0,43.0,47.0,13.0,24.0,296.11,23.65,23.03,24.21,23.56,284.0,